# Identified dataset : world university ranking

In [1]:
# imports
import pandas as pd 

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
import os


The university dataset has many .csv files.

For this experiment we will be exploring only the cwurData.csv and timesData.csv

In [2]:
def loadCSVData(path):
    '''
        Function helps load the given path name if it is a CSV file into a DataFrame using pandas
    '''
    if os.path.splitext(path)[1] == ".csv": # read only csv files from the dataset
        df = pd.read_csv(path, delimiter=',')

        return df
    return None

In [3]:
# file cwurData.csv
currUniData = loadCSVData("../world_university_ranking/cwurData.csv")
print(currUniData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   world_rank            2200 non-null   int64  
 1   institution           2200 non-null   object 
 2   country               2200 non-null   object 
 3   national_rank         2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  score                 2200 non-null   float64
 13  year                  2200 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 240.8+ KB
None


In [4]:
currUniData.isnull().sum()

world_rank                0
institution               0
country                   0
national_rank             0
quality_of_education      0
alumni_employment         0
quality_of_faculty        0
publications              0
influence                 0
citations                 0
broad_impact            200
patents                   0
score                     0
year                      0
dtype: int64

In [5]:
# comparative task 1 : compare score of top 5 universities over years from 2012 - 2015

def filter_scores(df : pd.DataFrame, uni_names : list[str]) -> pd.DataFrame :  
    return df.query(f"institution in {uni_names}")

uni_names = list(currUniData.head(5)["institution"]) # top 5 university for year 2012

workingData = filter_scores(currUniData, uni_names)
# print(workingData)

fig = px.bar(workingData, x="year", y="score", color="institution", barmode="group",
        title="Comparision of scores between the top five university from 2012 - 2015", text="score")
fig.show(renderer="browser", auto_open=True)

In [6]:
# composition task 1
# top 20 countries with most universities in year 2015
# used file : cwurdata.csv
year = 2015

def getUniCount(df : pd.DataFrame) -> pd.DataFrame:
    countryCount = {}

    for ind, row in df.iterrows():
        cname = row["country"]

        if cname in countryCount:
            countryCount[cname] += 1
        else:
            countryCount[cname] = 1
    
    countDf = pd.DataFrame(countryCount.items(), columns=['Country', 'Number of university']) # create a dataframe
    return countDf.sort_values(by=["Number of university"], ascending=False) # sort by number of university in descending order

countData = getUniCount(currUniData[currUniData["year"] == year])
countData = countData.iloc[:20, :] # get the first 20 countries

fig = px.bar(countData, x="Country", y="Number of university",
        color = "Number of university", color_continuous_scale='magenta',
        text = "Number of university",
        title = f"Top 20 countires with most universities in the year {year}"
    )

fig.update_layout(title_x = 0.5)
        
fig.show(renderer="browser", auto_open=True)    

Using another dataset timesData.csv for other tasks

In [7]:
# loading timesData.csv
timesUniData = loadCSVData("../world_university_ranking/timesData.csv")

# info on columns
print(timesUniData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2603 non-null   object 
 1   university_name         2603 non-null   object 
 2   country                 2603 non-null   object 
 3   teaching                2603 non-null   float64
 4   international           2603 non-null   object 
 5   research                2603 non-null   float64
 6   citations               2603 non-null   float64
 7   income                  2603 non-null   object 
 8   total_score             2603 non-null   object 
 9   num_students            2544 non-null   object 
 10  student_staff_ratio     2544 non-null   float64
 11  international_students  2536 non-null   object 
 12  female_male_ratio       2370 non-null   object 
 13  year                    2603 non-null   int64  
dtypes: float64(4), int64(1), object(9)
memor

In [8]:
timesUniData.isnull().sum()

world_rank                  0
university_name             0
country                     0
teaching                    0
international               0
research                    0
citations                   0
income                      0
total_score                 0
num_students               59
student_staff_ratio        59
international_students     67
female_male_ratio         233
year                        0
dtype: int64

In [9]:
before = timesUniData.shape[0]
timesUniData = timesUniData.dropna()

print(f"Dropped {before - timesUniData.shape[0]} NaN values")
print(timesUniData.isnull().sum())

Dropped 241 NaN values
world_rank                0
university_name           0
country                   0
teaching                  0
international             0
research                  0
citations                 0
income                    0
total_score               0
num_students              0
student_staff_ratio       0
international_students    0
female_male_ratio         0
year                      0
dtype: int64


In [10]:
def parseNumberOfStudents(df : pd.Series) -> list[float]:
    if df.dtype == object:
        tmp = [item.replace(',', '') for ind, item in df.iteritems() ]
        return list(map(float, tmp))
    else:
        return list(df)

def parseGenderRatio(df : pd.Series) -> list[float]:
    lst = []
    ratio = 0

    if df.dtype == object:
        for ind, item in df.iteritems():
            try:
                tmp = list(map(int, item.split(":") ) )
                lst.append(tmp[0]) 
            except ZeroDivisionError:
                lst.append(100)
            except:
                lst.append(0) # no data

        return lst
    else:
        return list(df)

In [11]:
# here since number of students and gender ratio is given in strings we need to
# convert them into numeric types

num_stud = parseNumberOfStudents(timesUniData.num_students)
timesUniData.num_students = num_stud
pd.to_numeric(timesUniData.num_students)

gender_ratio = parseGenderRatio(timesUniData.female_male_ratio)
timesUniData.female_male_ratio = gender_ratio
pd.to_numeric(timesUniData.female_male_ratio)

print(timesUniData.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2362 entries, 1 to 2602
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   world_rank              2362 non-null   object 
 1   university_name         2362 non-null   object 
 2   country                 2362 non-null   object 
 3   teaching                2362 non-null   float64
 4   international           2362 non-null   object 
 5   research                2362 non-null   float64
 6   citations               2362 non-null   float64
 7   income                  2362 non-null   object 
 8   total_score             2362 non-null   object 
 9   num_students            2362 non-null   float64
 10  student_staff_ratio     2362 non-null   float64
 11  international_students  2362 non-null   object 
 12  female_male_ratio       2362 non-null   int64  
 13  year                    2362 non-null   int64  
dtypes: float64(5), int64(2), object(7)
memor

In [12]:
# comparision task 2
# polar chart of teaching, international outlook, research, citations, income, gender ratio for top 3 universities
# in the year 2016
year = 2016

uni2016 = timesUniData[timesUniData["year"] == year]
topUnis = uni2016.iloc[:3,:]
print(topUnis)
goScatterData = [] # to store scatter plots data

for ind, row in topUnis.iterrows():
    goScatterData.append(go.Scatterpolar(
        r = [row.teaching, row.international, row.research, row.citations, 
                row.income, row.female_male_ratio ],
        theta = ["teaching", "International popularity", "Research", "Citations", "Income", "Female to male ratio"],
        name = row.university_name,
        fill='toself'
    ))

layout = dict(
    title = 'Polar chart of teaching, research, citations, income, gender ratio of the top 3 University for the year 2016',
    legend_title = 'University name'
)

fig = go.Figure(data = goScatterData, layout=layout)
fig.show(renderer="browser", auto_open=True)    

     world_rank                     university_name                   country  \
1803          1  California Institute of Technology  United States of America   
1804          2                University of Oxford            United Kingdom   
1805          3                 Stanford University  United States of America   

      teaching international  research  citations income total_score  \
1803      95.6          64.0      97.6       99.8   97.8        95.2   
1804      86.5          94.4      98.9       98.8   73.1        94.2   
1805      92.5          76.3      96.2       99.9   63.3        93.9   

      num_students  student_staff_ratio international_students  \
1803        2243.0                  6.9                    27%   
1804       19919.0                 11.6                    34%   
1805       15596.0                  7.8                    22%   

      female_male_ratio  year  
1803                 33  2016  
1804                 46  2016  
1805                 42  

In [13]:
# composition task 2
# Pie chart showing the number of students of the top 10 university for the year 2015
year = 2015

uni2015 = timesUniData[timesUniData["year"] == year]
topUnis = uni2015.iloc[:10, :]


goPieData = go.Pie(
    values = topUnis.num_students, 
    labels = topUnis.university_name,
    hole = .25,
    textinfo = 'label+percent',
)

layout = dict(
    title = f"Number of students studying in the top 10 univeristy - {year}",
    title_x = 0.3,
    legend_title_text ='University names'
)

fig = go.Figure(data = goPieData, layout=layout)
fig.show(renderer="browser", auto_open=True)   

In [14]:
# comparative task 3
# compare the Research and income of the top 10 university for the year of 2015
year = 2015
num_uni = 7

topUnis = timesUniData[timesUniData["year"] == year].head(num_uni)
topUnis = topUnis.sort_values(by=["income"], ascending=True)

fig = make_subplots(rows=1, cols=2, vertical_spacing=0.001,
    specs=[[{}, {}]], subplot_titles=("Research", "Income"))

    
researchPlot = go.Bar(y = topUnis.university_name, x = topUnis.research,
        name = 'Research', orientation='h',
        marker = dict(
            color = 'lightblue',
            line = dict(
                color = 'gray',
                width = 2
            )
        )
    )

incomePlot = go.Bar(y = topUnis.university_name, x = topUnis.income,
        name = 'Income', orientation ='h',
        marker = dict(
            color = '#eda882',
            line = dict(
                color = 'gray',
                width = 2
            )
        ),
    )

fig.append_trace(researchPlot, 1, 1)
fig.append_trace(incomePlot, 1, 2)
fig.update_yaxes(showticklabels=False, row=1, col=2)

fig.update_layout(title = f"Research and income of the top {num_uni} universities - {year}",
    title_x = .5,
    legend = dict(
        orientation = "h",
        yanchor ="bottom",
        y = 1.02,
        xanchor = "right",
        x = 1
    ),
)

fig.show(renderer="browser", auto_open=True)   

In [15]:
# composition task 3
# Stacked bar graph of the female to male ratio of the top 5 university in the USA for the time span of 2011-2016
year_s = 2011
year_e = 2016
num_uni = 5

def getWorkingData(df : pd.DataFrame) -> pd.DataFrame:
    lst = [] # empty list

    for i in range(year_s, year_e + 1):
        # since there are five Universities, we have to normalize each indivisual data
        # by dividing it by number of universities

        tmpDf = df[df["year"] == i].head(num_uni) # get top 5university for a specific year
        tmpDf.female_male_ratio = tmpDf.female_male_ratio / num_uni 

        lst.append(tmpDf)
    
    return pd.concat(lst)

uniUK = timesUniData[timesUniData["country"] == "United Kingdom"]

topUnisByYear = getWorkingData(uniUK)

fig = px.bar(topUnisByYear, x="year", y="female_male_ratio", color="university_name",  text="female_male_ratio",
        labels= {
            'female_male_ratio' : "Female to male ratio",
            'year' : 'Year',
            "university_name" : "University name",
        })

fig.update_layout(
    title = f"Stacked bar graph of the female-to-male ratio of the top 5 university in the UK for the time span of {year_s}-{year_e}"
)

fig.show(renderer="browser", auto_open=True)   

In [16]:
# comparative task 4
# here we use cwurData.csv dataset
# stacked chart comparing quality of education of the top 5 university for the year 2012 - 2015

year_s = 2012
year_e = 2015
num_uni = 5

def getIndivisualData(df : pd.DataFrame, uniNames : list[str]) -> dict[str, pd.DataFrame] :
    dataList = {}

    for name in uniNames:
        dataList[name] = df[df["institution"] == name]

    return dataList

uni2012 = currUniData[currUniData["year"] == year_s]["institution"]

# we are excluding havard as its data is all ones when compared to others
uni_names = list(uni2012.iloc[1 : (num_uni + 1)] ) 

workingData = getIndivisualData(currUniData, uni_names)

goScatterData = []
fillMode = 'tonexty'

for uName, data in workingData.items():
    goScatterData.append(go.Scatter(
            x = data.year,
            y = data.quality_of_education,
            fill = fillMode,
            mode = 'lines+markers',
            name = uName,
            hovertemplate = f"<b> University Name </b> = {uName}" +
                "<br><b> Year </b> = %{x}" +
                "<br><b> Quality of Education </b> = %{y}" 
        )
    )

layout = dict(
    title = f"Quality of Education of the top 5 Universities for the year {year_s} to  {year_e}",
    title_x = 0.2,
    xaxis = dict(
        tickmode = 'linear',
        tick0 = year_s - 1,
        dtick = 1
    ),
    xaxis_title="Year",
    yaxis_title="Quality of Education",
    legend_title="University name"
)

fig = go.Figure(data = goScatterData, layout = layout)
fig.show(renderer="browser", auto_open=True)  

In [ ]:
# composition task 4
# top 20 countries with universities having the most influence

year = 2015

def getUniInfluence(df : pd.DataFrame) -> pd.DataFrame:
    countryInflucence = {}

    for ind, row in df.iterrows(): 
        cname = row["country"]

        if cname in countryInflucence:
            countryInflucence[cname] += row.influence
        else:
            countryInflucence[cname] = row.influence

    # create a dataframe
    countDf = pd.DataFrame(countryInflucence.items(), columns=['Country', 'Influcence of Universities']) 

    # sort by influence descending order
    return countDf.sort_values(by=["Influcence of Universities"], ascending=False) 

influenceData = getUniInfluence(currUniData[currUniData["year"] == year])
influenceData = influenceData.iloc[:20, :] # get the first 20 countries

fig = px.bar(influenceData, x="Country", y="Influcence of Universities",
        color = "Influcence of Universities", color_continuous_scale='bluered',
        text = "Influcence of Universities",
        title = f"Top 20 countires with universities having most influence - {year}"
    )

fig.update_layout(title_x = 0.5)
        
fig.show(renderer="browser", auto_open=True)   